# Algorytmy tekstowe - laboraorium 3

### Importowanie bibliotek:

In [98]:
import numpy as np
import spacy
from random import randint

### 1. Zaimplementuj algorytm obliczania odległości edycyjnej w taki sposób, aby możliwe było określenie przynajmniej jednej sekwencji edycji (dodanie, usunięcie, zmiana znaku), która pozwala w minimalnej liczbie kroków, przekształcić jeden łańcuch w drugi.

In [15]:
def delta(a, b):
    if a == b: return 0
    else: return 1
    
def edit_distance(s1, s2):
    m, n = len(s1), len(s2)
    dp = [[0 for _ in range(n+1)] for _ in range(m+1)]
    
    for i in range(1,m+1): 
        dp[i][0] = i
    for j in range(1, n+1):
        dp[0][j] = j
    
    for i in range(1,m+1):
        for j in range(1,n+1):
            if s1[i-1] == s2[j-1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                dp[i][j] = min(dp[i][j-1] + 1, dp[i-1][j] + 1, dp[i-1][j-1] + delta(s1[i-1], s2[j-1]))
                
    return dp

In [16]:
"Edit distance for (wojtk, wjeek) : " + str(edit_distance("wojtk", "wjeek")[-1][-1])

'Edit distance for (wojtk, wjeek) : 3'

### 2. Na podstawie poprzedniego punktu zaimplementuj prostą wizualizację działania algorytmu, poprzez wskazanie kolejnych wersji pierwszego łańcucha, w których dokonywana jest określona zmiana. "Wizualizacja" może działać w trybie tekstowym. Np. zmiana łańcuch "los" w "kloc" może być zrealizowana następująco: klos (dodanie litery k) kloc (zamiana s->c)

In [76]:
def printer(s1, s2, dp):
    m, n = len(s1), len(s2)
    print("Before: " + s1)
    print("After: " + s2)
    print("---------------------------------")
    print("--------------------------------- \n")
    substitute_str = "REPLACE:  {}  ->  {}"
    delete_str = "REMOVE:   {}  ->  {}"
    insert_str = "INSERT:   {}  ->  {}"

    while m > 0 and n > 0:
        
        if s1[m-1] != s2[n-1]:
            substitute = dp[m-1][n-1] + delta(s1[m-1], s2[n-1])
            delete = dp[m-1][n] + 1
            
            if dp[m][n] == substitute:
                operation = substitute_str.format(s1[m-1], s2[n-1])
                s1 = s1[:m-1] + s2[n-1] + s1[m:]
                m -= 1
                n -= 1
            elif dp[m][n] == delete:
                operation = delete_str.format(s1[m-1], s2[n-1:])
                s1 = s1[:m-1] + s1[m:]
                m -= 1
            else:
                operation = insert_str.format(s2[n-1], s1[m:])
                s1 = s1[:m] + s2[n-1] + s1[m:]
                n -= 1
                
            print(operation)
            print("NOW: {}".format(s1))
            print("---------------------------------")
            
        else:
            m, n = m-1, n-1

    while m > 0:
        operation = delete_str.format(s1[m-1], s2[n:])
        s1 = s1[:m-1] + s1[m:]
        print(operation)
        print("NOW: {}".format(s1))
        print("---------------------------------")
        m -= 1

    while n > 0:
        operation = insert_str.format(s2[n-1], s1[m:])
        s1 = s1[:m] + s2[n-1] + s1[m:]
        print(operation)
        print("NOW: {}".format(s1))
        print("---------------------------------")
        n -= 1
        
    print("\nEquals? " + str(s1==s2))
        
def visualization(s1, s2):
    dp = edit_distance(s1, s2)
    printer(s1,s2,dp)

### 3. Przedstaw wynik działania algorytmu z p. 2 dla następujących par łańcuchów:

In [77]:
visualization("los", "kloc")

Before: los
After: kloc
---------------------------------
--------------------------------- 

REPLACE:  s  ->  c
NOW: loc
---------------------------------
INSERT:   k  ->  loc
NOW: kloc
---------------------------------

Equals? True


In [78]:
visualization("Łódź", "Lodz")

Before: Łódź
After: Lodz
---------------------------------
--------------------------------- 

REPLACE:  ź  ->  z
NOW: Łódz
---------------------------------
REPLACE:  ó  ->  o
NOW: Łodz
---------------------------------
REPLACE:  Ł  ->  L
NOW: Lodz
---------------------------------

Equals? True


In [79]:
visualization("kwintesencja", "quintessence")

Before: kwintesencja
After: quintessence
---------------------------------
--------------------------------- 

REPLACE:  a  ->  e
NOW: kwintesencje
---------------------------------
REMOVE:   j  ->  ce
NOW: kwintesence
---------------------------------
INSERT:   s  ->  sence
NOW: kwintessence
---------------------------------
REPLACE:  w  ->  u
NOW: kuintessence
---------------------------------
REPLACE:  k  ->  q
NOW: quintessence
---------------------------------

Equals? True


In [80]:
visualization("ATGAATCTTACCGCCTCG", "ATGAGGCTCTGGCCCCTG")

Before: ATGAATCTTACCGCCTCG
After: ATGAGGCTCTGGCCCCTG
---------------------------------
--------------------------------- 

REMOVE:   C  ->  TG
NOW: ATGAATCTTACCGCCTG
---------------------------------
REPLACE:  G  ->  C
NOW: ATGAATCTTACCCCCTG
---------------------------------
REPLACE:  C  ->  G
NOW: ATGAATCTTAGCCCCTG
---------------------------------
REPLACE:  A  ->  G
NOW: ATGAATCTTGGCCCCTG
---------------------------------
INSERT:   C  ->  TGGCCCCTG
NOW: ATGAATCTCTGGCCCCTG
---------------------------------
REPLACE:  T  ->  G
NOW: ATGAAGCTCTGGCCCCTG
---------------------------------
REPLACE:  A  ->  G
NOW: ATGAGGCTCTGGCCCCTG
---------------------------------

Equals? True


### 4. Zaimplementuj algorytm obliczania najdłuższego wspólnego podciągu dla pary ciągów elementów.

In [110]:
def LCS(A,B):
    n, m = len(A), len(B)
    dp = [[0] * (m+1) for _ in range(n+1)]
    
    for i in range(n):
        for j in range(m):
            if A[i] == B[j]:
                dp[i+1][j+1] = dp[i][j] + 1
            else:
                dp[i+1][j+1] = max(dp[i+1][j], dp[i][j+1])
                
    return dp

### 5. Korzystając z gotowego tokenizera (np spaCy - https://spacy.io/api/tokenizer) dokonaj podziału załączonego tekstu na tokeny.

In [100]:
nlp = spacy.load("pl_core_news_md")  
file_path = "romeo-i-julia-700.txt"
tokenizer = nlp.tokenizer
tokens = []

with open(file_path, encoding="utf-8") as f:
    text = f.read()
    tokens = tokenizer(text)

### 6. Stwórz 2 wersje załączonego tekstu, w których usunięto 3% losowych tokenów.

In [103]:
def remove_tokens(tokens):
    new = []
    for token in tokens:
        if randint(1, 100) > 3:
            new.append(token.text_with_ws)
    return new

new1 = remove_tokens(tokens)
new2 = remove_tokens(tokens)

with open("v1.txt", "w", encoding="utf-8") as f:
    for token in new1:
        f.write(token)

with open("v2.txt", "w", encoding="utf-8") as f:
    for token in new2:
        f.write(token)

### 7. Oblicz długość najdłuższego podciągu wspólnych tokenów dla tych tekstów. Traktujemy teraz token (wyraz) jako podstawową, niepodzielną jednostkę ciągu. 

In [105]:
print("Longest common subsequence length: " + str(LCS(new1, new2)))

Longest common subsequence length: 2542


### 8. Korzystając z algorytmu z punktu 4 skonstruuj narzędzie, o działaniu podobnym do narzędzia diff, tzn. wskazującego w dwóch plikach linie, które się różnią. Na wyjściu narzędzia powinny znaleźć się elementy, które nie należą do najdłuższego wspólnego podciągu. Należy wskazać skąd dana linia pochodzi (< > - pierwszy/drugi plik) oraz numer linii w danym pliku. Traktujemy teraz całą linię jako podstawową, niepodzielną jednostkę ciągu.

In [108]:
def diff(file1, file2):
    
    with open(file1, encoding="utf-8") as f1, open(file2, encoding="utf-8") as f2:
        text1 = f1.read().splitlines()
        text2 = f2.read().splitlines()

    dp = LCS(text1,text2)            
    diff_lines = []
    n, m = len(text1), len(text2)
    i, j = n, m
    
    while i > 0 and j > 0:
        if text1[i-1] == text2[j-1]:
            i, j = i-1, j-1
        elif dp[i-1][j] >= dp[i][j-1]:
            diff_lines.append(("<", i, text1[i-1]))
            i -= 1
        else:
            diff_lines.append((">", j, text2[j-1]))
            j -= 1
            
    while i > 0:
        diff_lines.append(("<", i, text1[i-1]))
        i -= 1
    while j > 0:
        diff_lines.append((">", j, text2[j-1]))
        j -= 1

    return reversed(diff_lines)

### 9. Przedstaw wynik działania narzędzia na tekstach z punktu 6. Zwróć uwagę na dodanie znaków przejścia do nowej linii, które są usuwane w trakcie tokenizacji.

In [111]:
result = diff("v1.txt", "v2.txt")
for line in result:
    print(line[0], line[1], line[2])

> 1 William Shakespeare
< 1 William 
> 11  * — książę panujący w Weronie
> 12  * PARYS — młody szlachetnego rodu, krewny księcia
< 11  * ESKALUS — książę panujący w Weronie
< 12  * PARYS — młody Weroneńczyk szlachetnego rodu, księcia
> 14  * STARZEC — stryjeczny brat Kapuleta
> 15  * ROMEO — syn Montekiego
> 16  * MERKUCJO — krewny księcia
< 14  * STARZEC — stryjeczny brat Kapuleta* ROMEO — syn Montekiego
< 15  * MERKUCJO — księcia
< 23  * APTEKARZ
< 24  TRZECH MUZYKANTÓW
< 25  * PAŹ PARYSA
> 25  * TRZECH 
> 26  * PAŹ PARYSA* PIOTR
> 27  * DOWÓDCA WARTY
> 28  * PANI MONTEKI — małżonka Montekiego
> 29  * PANI KAPULET — małżonka Kapuleta
< 27  * DOWÓDCA WARTY* PANI MONTEKI — małżonka Montekiego* PANI KAPULET małżonka Kapuleta
> 32  * Obywatele weroneńscy, różne osoby płci obojej, liczący się do przyjaciół obu domów, maski, straż wojskowa i inne osoby.
< 30  * Obywatele weroneńscy, różne osoby płci obojej, liczący się do przyjaciół obu domów, maski, straż wojskowa i osoby.
> 45 Dwa rody, 

> 623 testament: będzież to wezwanieDobre dla tego, kto jest tak złym stanie?
< 621 Pisać testament: będzie–ż to wezwanie
< 622 Dobre dla tego, kto jest w tak złym stanie?
> 635 Biegle celujesz. I ta, którą kocham,Jest piękna.
< 634 Biegle celujesz. I ta, którą kocham,
< 635 Jest piękna.
> 638 BENWOLIOW piękny trafić najłatwiej.
< 638 BENWOLIO
< 639 
< 640                         W piękny cel trafić najłatwiej.
> 643 A właśnieś chybił. Niczym tu kołczany
< 645 A właśnieś chybiłNiczym tu kołczany
> 648 Odpiera szturmy spojrzeń ;
< 650 Odpiera szturmy spojrzeń napastniczych;
> 651 Że kiedy umrze, do grobu z nią zstąpi
> 652 Całe bogactwo, którego tak skąpi.
< 653 Że kiedy umrze, do z nią zstąpi
< 654 Całe bogactwo, którego tak .
> 657 Wiecznie–ż chce sama zostać z swym bogactwem?ROMEO
< 659 Wiecznie–ż chce sama zostać z swym bogactwem?
> 659 Tak jest; i skąpstwo to jest ,
> 660 Bo piękność, którą własna srogość 
> 661 Całą potomność piękności pozbawia
> 662 Zbyt ona piękna, zbyt mądra za